- put you symbol, date range here, it will import data from nse and refine that data and save to excel/symbol.csv
- then perform 5-10 lac calculation and find what can be max profit, and save this final stp and trgt  to 'all_in_one.csv' file
- cumprod and cumsum will also me made in excel/symbol.csv file so that you can see in details

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from nselib import capital_market 
import pandas as pd

import matplotlib.pyplot as plt 
symbol = "SUZLON"
fdate='15-08-2022'
tdate='25-08-2024'


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from nselib import capital_market 
import pandas as pd

data = capital_market.price_volume_and_deliverable_position_data(symbol=symbol, from_date='15-08-2022', to_date='15-08-2024')
df = pd.DataFrame(data)
df1 = df[["Date", "Symbol", "OpenPrice", "HighPrice", "LowPrice", "ClosePrice"]]

# Function to replace commas only if the value is a string
def replace_commas(x):
    if isinstance(x, str):
        return x.replace(',', '')
    return x

# Apply the function to columns
df1["HighPrice"] = df1['HighPrice'].apply(replace_commas)
try:
    df1["OpenPrice"] = df1['OpenPrice'].apply(replace_commas)
    df1["ClosePrice"] = df1['ClosePrice'].apply(replace_commas)
    df1["LowPrice"] = df1['LowPrice'].apply(replace_commas)
except:
    print("Replace done!")

# Convert columns to numeric
df1["HighPrice"] = pd.to_numeric(df1["HighPrice"], errors='coerce')
df1["OpenPrice"] = pd.to_numeric(df1["OpenPrice"], errors='coerce')
df1["ClosePrice"] = pd.to_numeric(df1["ClosePrice"], errors='coerce')
df1["LowPrice"] = pd.to_numeric(df1["LowPrice"], errors='coerce')

# Calculate percentage changes
df1["High%"] = (df1["HighPrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
df1["High%"] = df1["High%"].apply(lambda x: "{:.3f}".format(x))

df1["Low%"] = (df1["LowPrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
df1["Low%"] = df1["Low%"].apply(lambda x: "{:.3f}".format(x))

df1["Close%"] = (df1["ClosePrice"] - df1["OpenPrice"]) * 100 / df1["OpenPrice"]
df1["Close%"] = df1["Close%"].apply(lambda x: "{:.3f}".format(x))

# Convert percentage columns back to numeric
df1["Low%"] = pd.to_numeric(df1["Low%"], errors='coerce')
df1["High%"] = pd.to_numeric(df1["High%"], errors='coerce')
df1["Close%"] = pd.to_numeric(df1["Close%"], errors='coerce')

# Calculate target and stoploss percentages
df1.loc[abs(df1['High%']) <= abs(df1['Low%']), 'target_wrt_h&l%'] = abs(df1['Low%'])
df1.loc[abs(df1['High%']) > abs(df1['Low%']), 'target_wrt_h&l%'] = abs(df1['High%'])
df1.loc[abs(df1['High%']) >= abs(df1['Low%']), 'stoploss_wrt_h&l%'] = abs(df1['Low%'])
df1.loc[abs(df1['High%']) < abs(df1['Low%']), 'stoploss_wrt_h&l%'] = abs(df1['High%'])

# Determine profit decider
df1.loc[abs(df1['target_wrt_h&l%']) == abs(df1['High%']), 'profit_decider'] = abs(df1['Low%'])
df1.loc[abs(df1['target_wrt_h&l%']) == abs(df1['Low%']), 'profit_decider'] = abs(df1['High%'])

# Fill missing values
df1 = df1.fillna(0)

# Convert date columns and extract date parts
df1['Date'] = pd.to_datetime(df1['Date'], format='%d-%b-%Y')
df1['Day'] = df1['Date'].dt.day
df1['Month'] = df1['Date'].dt.month
df1['Year'] = df1['Date'].dt.year
df1['profit_decider'] = pd.to_numeric(df1['profit_decider'], errors='coerce')
df1.to_csv(f"excel/{symbol}.csv",index=False)
print("done!")


In [9]:
def prfdata(symbol,stploss,trgt):
    df1=pd.read_csv(f"excel/{symbol}.csv")
   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])
    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])

    profit = df1['profit'].sum()
    # print(f"Total profit = {profit}% in 2 years")
    data1 = df1.groupby(['Year', 'Month']).agg({"profit": "sum"})

    data = 1 + data1.values / 100
    overallprofit = 100 * data.cumprod()[-1:][0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    # print(f"Total profit made in 2 years = {formatted_overallprofit}%")

    arr=[]

    for i in range(len(data1.values)):
        n=data1.values[i][0]
        arr.append(n)
    dic={"tprofit":profit,"cprofit":formatted_overallprofit,"data":data1}
    # df1.to_csv(f"excel/{symbol}.csv",index=False)
    
    return dic


In [ ]:
def prfdata1(symbol,stploss,trgt):
    df1=pd.read_csv(f"excel/{symbol}.csv")
   
    df1['stploss']=df1['target_wrt_h&l%'].apply(lambda x:stploss)
    df1['trgt']=df1['target_wrt_h&l%'].apply(lambda x:trgt)
    df1['stploss']=pd.to_numeric(df1['stploss'])
    df1['trgt']=pd.to_numeric(df1['trgt'])
    df1.loc[((abs(df1['target_wrt_h&l%']) >= df1['trgt']) & ((abs(df1['profit_decider'])) < df1['stploss'])), 'profit'] = (df1['trgt']-df1['stploss'])
    df1.loc[(abs(df1['profit_decider']) >= df1['stploss']) , 'profit'] = -(2 * df1['stploss'])
    df1.loc[((abs(df1['target_wrt_h&l%']) < abs(df1['trgt'])) & ((abs(df1['profit_decider'])) < df1['stploss']) ) , 'profit'] = (abs(df1['Close%'])-df1['stploss'])
    df1['profit'] = pd.to_numeric(df1['profit'])

    profit = df1['profit'].sum()
    # print(f"Total profit = {profit}% in 2 years")
    data1 = df1.groupby(['Year', 'Month']).agg({"profit": "sum"})

    data = 1 + data1.values / 100
    overallprofit = 100 * data.cumprod()[-1:][0]
    formatted_overallprofit = "{:.4f}".format(overallprofit)
    # print(f"Total profit made in 2 years = {formatted_overallprofit}%")

    arr=[]

    for i in range(len(data1.values)):
        n=data1.values[i][0]
        arr.append(n)
    dic={"tprofit":profit,"cprofit":formatted_overallprofit,"data":data1}
    df1.to_csv(f"excel/{symbol}.csv",index=False)
    
    return dic


In [ ]:
stp = np.arange(0,10,0.01)
trgt =np.arange(0,30,0.1)
arr=[]
max=0
fstp=0
ftrgt=0
pmdata=[]
for estp in stp:
    print(estp)
    for etrgt in trgt:
        cc=prfdata(symbol,estp,etrgt)
        arr.append(cc['tprofit'])
        
        if(max<cc['tprofit']):            
            max=cc['tprofit']
            fstp=estp
            ftrgt=etrgt
            pmdata=cc['data']
            # print(f"{max}  {estp}  {etrgt} ")
prfdata1(symbol,fstp,ftrgt)
        
            


In [11]:
maxc=(np.array(pmdata)/100+1).cumprod()[-1:][0]*100

In [12]:

dataf={"symbol":[symbol],"tprofit":[max],"cprofit":[maxc],"target":[ftrgt],"stoploss":[fstp]}
df23=pd.DataFrame(dataf)
df24=pd.read_csv("all_in_one.csv")
df25=pd.concat([df24,df23])
df25.to_csv("all_in_one.csv",index=False)

In [ ]:
dic=prfdata(symbol,fstp,ftrgt)
print("total profit=",dic['tprofit'])
print("cumprod profit=",dic['cprofit'])
dic['data']


In [14]:

ddata=pd.read_csv(f"excel/{symbol}.csv")
ddata["cumsum"]=ddata['profit'].cumsum()
ddata["cumprod"]=(ddata['profit']/100+1).cumprod()
ddata.to_csv(f"excel/{symbol}.csv")